In [ ]:
from prophet import Prophet
import pmdarima as pm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
#from fbprophet import Prophet
#from fbprophet.plot import plot_plotly, add_changepoints_to_plot
# from keras.models import Sequential
# from keras.layers import Dense

# from keras.optimizers import Adam
import prophet
warnings.filterwarnings('ignore')

In [ ]:
czech  = pd.read_csv('univariate_models/univariate_df_Czechia.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
eston  = pd.read_csv('univariate_models/univariate_df_Estonia.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
franc  = pd.read_csv('univariate_models/univariate_df_France.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
malay  = pd.read_csv('univariate_models/univariate_df_Malaysia.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
belgi  = pd.read_csv('univariate_models/univariate_df_Belgium.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
chile  = pd.read_csv('univariate_models/univariate_df_Chile.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
irela  = pd.read_csv('univariate_models/univariate_df_Ireland.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
israe  = pd.read_csv('univariate_models/univariate_df_Israel.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
italy  = pd.read_csv('univariate_models/univariate_df_Italy.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)
us  = pd.read_csv('univariate_models/univariate_df_United_States.csv',usecols=[1,2],  parse_dates=[0]).reset_index(drop=True)

#sort index
for x in [czech, eston, franc, malay,belgi, chile, irela, israe, italy, us]:
    x = x.sort_index()
train_size = int(len(czech) * 0.8)
czech_train, czech_test = czech[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), czech[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
eston_train, eston_test = eston[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), eston[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
franc_train, franc_test = franc[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), franc[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
malay_train, malay_test = malay[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), malay[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
belgi_train, belgi_test = belgi[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), belgi[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
chile_train, chile_test = chile[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), chile[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
irela_train, irela_test = irela[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), irela[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
israe_train, israe_test = israe[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), israe[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
italy_train, italy_test = italy[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), italy[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
us_train, us_test = us[0:train_size].rename(columns={'date': 'ds', 'new_deaths': 'y'}), us[train_size:].rename(columns={'date': 'ds', 'new_deaths': 'y'})
czech_train

In [ ]:
def mean_absolute_scaled_error(y_true, y_pred):
    naive_forecast = y_true[:-1]
    y_true_insample = y_true[1:]
    insample_error = np.abs(y_true_insample - naive_forecast)
    # Calculate the forecast errors
    forecast_error = np.abs(y_true - y_pred)
    # Calculate the MASE
    mase = np.mean(forecast_error) / np.mean(insample_error)
    return mase

metrics_df = pd.DataFrame(columns=['Model', 'MAE', 'MASE'])

In [ ]:
train, test = czech_train, czech_test
model_baseline = Prophet()
model_baseline.fit(train)
future_baseline = model_baseline.make_future_dataframe(periods=41) # periods are 41